In [74]:
import pandas as pd
import numpy as np
import math

import panel as pn #Library for panel
import panel.widgets as pnw #Library for widgets
from ipywidgets import widgets
import plotly.graph_objects as go
pn.extension('plotly')


In [66]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# FUNCTIONS

In [76]:
###############################################################################################
# Brownian Motion Trajectory 3D
###############################################################################################

#Definition of the function
def bm_3d(n_steps, speed, s_x_pos, s_y_pos):

    #Initialization of velocity vector
    velocity = Vec2d(speed,0)
    time_elapsed = 0
    BM_3d_df = pd.DataFrame(columns = ['x_pos','y_pos','z_pos']) #Definition of the data frame

    temp_df = pd.DataFrame([{'x_pos': s_x_pos, 'y_pos': s_y_pos, 'z_pos': time_elapsed}])
    BM_3d_df = pd.concat([BM_3d_df, temp_df], ignore_index = True)

    for i in range(n_steps-1):  
        #Chosse a random number from 0, 1.57, 3.1416, 4.7123
        turn_angle = np.random.choice([
                                        0, #0
                                        np.pi/2, #1.5707
                                        np.pi, #3.1416
                                        3*np.pi/2 # 3pi/2 = 4.7123
                                      ])
        
        time_elapsed= time_elapsed + 0.1
        
        velocity = velocity.rotated(turn_angle)
        temp_df = pd.DataFrame([{
                                    'x_pos': BM_3d_df.x_pos[i]+velocity.x, 
                                    'y_pos': BM_3d_df.y_pos[i]+velocity.y,
                                    'z_pos': time_elapsed }])

        BM_3d_df = pd.concat([BM_3d_df, temp_df], ignore_index=True)
    
    return BM_3d_df



## Random Correlated Walk

In [77]:
def generate_correlated_random_walk_3d(num_steps, correlation_coefficient):

    # Inicializar el DataFrame para las posiciones
    df = pd.DataFrame(columns=['X', 'Y', 'Z'])
    delta_t = 0.1
    # Inicializar las posiciones
    positions = np.zeros((num_steps + 1, 3))
    
    # Inicializar la dirección previa
    prev_direction = np.random.normal(size=3)
    
    for i in range(1, num_steps + 1):
        # Generar un nuevo paso con correlación
        new_direction = (correlation_coefficient * prev_direction + 
                         np.random.normal(size=3) * np.sqrt(1 - correlation_coefficient**2))
        new_direction /= np.linalg.norm(new_direction)  # Normalizar el vector de dirección
        
        # Calcular el incremento
        increment = new_direction * np.sqrt(delta_t)
        
        # Actualizar la posición
        positions[i] = positions[i - 1] + increment
        
        # Actualizar la dirección previa
        prev_direction = new_direction
    
    # Guardar las posiciones en el DataFrame
    df = pd.DataFrame(positions, columns=['X', 'Y', 'Z'])
    
    return df


## Levy Flight

In [78]:
def levy_flight_step(alpha, size=3):
    # Generar una dirección aleatoria
    direction = np.random.normal(size=size)
    direction /= np.linalg.norm(direction)  # Normalizar la dirección

    # Generar la longitud del paso según la distribución de Lévy
    step_length = np.random.pareto(alpha)
    
    # Calcular el paso
    step = direction * step_length
    return step

def generate_levy_flight_3d(num_steps, alpha):

    # Inicializar el DataFrame para las posiciones
    df = pd.DataFrame(columns=['X', 'Y', 'Z'])
    
    # Inicializar las posiciones
    positions = np.zeros((num_steps + 1, 3))
    
    for i in range(1, num_steps + 1):
        # Generar un nuevo paso de Levy flight
        step = levy_flight_step(alpha)
        
        # Actualizar la posición
        positions[i] = positions[i - 1] + step
    
    # Guardar las posiciones en el DataFrame
    df = pd.DataFrame(positions, columns=['X', 'Y', 'Z'])
    
    return df


In [118]:
# Datos de ejemplo
n_steps = 1000 #Number of steps
speed = 6 #Speed
s_x_pos = 0 #Start position x var
s_y_pos = 0 #Start position y var
correlation_coefficient = 0.9  # Coeficiente de correlación entre 0 y 1
alpha = 1.5       # Parámetro de la distribución de Lévy (0 < alpha < 2)


trajectory_BM_3d = bm_3d(n_steps, speed, s_x_pos, s_y_pos)
trajectory_LF_3d = generate_levy_flight_3d(n_steps, alpha)
trajectory_CRW_3d = generate_correlated_random_walk_3d(n_steps, correlation_coefficient)

# Crear la figura
fig = go.Figure()


fig.add_trace(go.Scatter3d(
    x=trajectory_BM_3d['x_pos'],
    y=trajectory_BM_3d['y_pos'],
    z=trajectory_BM_3d['z_pos'],
    mode='lines',
    line=dict(width=2),
    name='BM'
))

fig.add_trace(go.Scatter3d(
    x=trajectory_LF_3d['X'],
    y=trajectory_LF_3d['Y'],
    z=trajectory_LF_3d['Z'],
    mode='lines',
    line=dict(width=2),
    name = 'CRW'
))

fig.add_trace(go.Scatter3d(
    x=trajectory_CRW_3d['X'],
    y=trajectory_CRW_3d['Y'],
    z=trajectory_CRW_3d['Z'],
    mode='lines',
    line=dict(width=2),
    name = 'LF'
))



# Actualizar el diseño para incluir los checkboxes
fig.update_layout(
    title='Gráfico con Checkboxes en Plotly',
    updatemenus=
    [{
        'buttons': [
            {
                'label': 'Brownian Motion',
                'method': 'update',
                'args': [{'visible': [True, False, False]}, {'title': 'Serie 1 Visible'}]
            },
            {
                'label': 'Correlated Random Walk',
                'method': 'update',
                'args': [{'visible': [False, True, False]}, {'title': 'Serie 2 Visible'}]
            },
            {
                'label': 'Levi fligths',
                'method': 'update',
                'args': [{'visible': [False, False, True]}, {'title': 'Serie 3 Visible'}]
            },
            {
                'label': 'Mostrar Todo',
                'method': 'update',
                'args': [{'visible': [True, True, True]}, {'title': 'Todas las Series'}]
            },
            {
                'label': 'Ocultar Todo',
                'method': 'update',
                'args': [{'visible': [False, False, False]}, {'title': 'Ninguna Serie Visible'}]
            }
        ],
        'direction': 'down',
        'showactive': True,

    }],
    xaxis_title='X',
    yaxis_title='Y',
    margin=dict(l=0, r=0, b=0, t=40)


)

# Mostrar el gráfico
fig.show()


In [120]:
# Datos de ejemplo
#n_steps = 1000 #Number of steps
#speed = 6 #Speed
#s_x_pos = 0 #Start position x var
#s_y_pos = 0 #Start position y var
correlation_coefficient = 0.9  # Coeficiente de correlación entre 0 y 1
alpha = 1.5       # Parámetro de la distribución de Lévy (0 < alpha < 2)

n_steps = pnw.IntInput(name='number of steps', value=50, step=5, start=1, end=1000)
speed   = pnw.IntInput(name='speed', value=1, step=1, start=1, end=100)
s_x_pos = pnw.IntInput(name='x starting pos', value=0, step=1, start=-100, end=100)
s_y_pos = pnw.IntInput(name='y starting pos', value=0, step=1, start=-100, end=100)
#correlation_coefficient = pnw.IntInput(name='correlation coefficient', value=0, step=1,start=0, end=1)
#alpha = pnw.IntInput(name='alpha',value=0, step_=1, start=0,end =2 )

@pn.depends(n_steps, speed, s_x_pos, s_y_pos)
def plot_traj(n_steps, speed, s_x_pos, s_y_pos):
    #rw_bm_df = bm_2d(n_steps, speed, s_x_pos=s_x_pos, s_y_pos=s_y_pos)
    trajectory_BM_3d = bm_3d(n_steps, speed, s_x_pos, s_y_pos)
    trajectory_LF_3d = generate_levy_flight_3d(n_steps, alpha)
    trajectory_CRW_3d = generate_correlated_random_walk_3d(n_steps, correlation_coefficient)

# Crear la figura
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=trajectory_BM_3d['x_pos'],
        y=trajectory_BM_3d['y_pos'],
        z=trajectory_BM_3d['z_pos'],
        mode='lines',
        line=dict(width=2),
        name='BM'
    ))

    fig.add_trace(go.Scatter3d(
        x=trajectory_LF_3d['X'],
        y=trajectory_LF_3d['Y'],
        z=trajectory_LF_3d['Z'],
        mode='lines',
        line=dict(width=2),
        name = 'LF'
    ))

    fig.add_trace(go.Scatter3d(
        x=trajectory_CRW_3d['X'],
        y=trajectory_CRW_3d['Y'],
        z=trajectory_CRW_3d['Z'],
        mode='lines',
        line=dict(width=2),
        name = 'CRW'
    ))
    
    # Add Buttons
    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                direction="right",
                active=0,
                x=0.57,
                y=1.2,
                buttons=list([
                    dict(label="BM",
                         method="update",
                         args=[{"visible": [True, False, False]},
                               {"title": "BM",
                                }]),
                    dict(label="LF",
                         method="update",
                         args=[{"visible": [False,True,False]},
                               {"title": "LF",
                                }]),
                    dict(label="CRW",
                         method="update",
                         args=[{"visible": [False, False, True]},
                               {"title": "CRW",
                                }])               
                ]),
            )
        ],
        xaxis_title='X',
        yaxis_title='Y',
        margin=dict(l=0, r=0, b=0, t=40)
    )
    return fig


# Mostrar el gráfico
pn.Row(pn.Column(n_steps,speed, s_x_pos, s_y_pos), plot_traj)


Row
    [0] Column
        [0] IntInput(end=1000, name='number of steps', start=1, step=5, value=50)
        [1] IntInput(end=100, name='speed', start=1, value=1)
        [2] IntInput(end=100, name='x starting pos', start=-100)
        [3] IntInput(end=100, name='y starting pos', start=-100)
    [1] ParamFunction(function)